## Wordnet
#### Sources
* [Blog](https://blog.xrds.acm.org/2017/07/power-wordnet-use-python/#:~:text=WordNet%20is%20a%20database%20of,and%20sat%20on%20the%20sofa.%E2%80%9D)
* [Theory](https://wordnet.princeton.edu/)
* [Medium article on query relaxation](https://queryunderstanding.com/query-relaxation-342bc37ad425)
* [ Paper: Similarity Search Combining Query Relaxation and Diversification](https://arxiv.org/ftp/arxiv/papers/1611/1611.04689.pdf)
* [Similarity application example](https://stackoverflow.com/questions/15730473/wordnet-find-synonyms)

In [1]:
from nltk.corpus import wordnet as wn
import pandas as pd
import re
import pickle
import os
import string
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import operator
import itertools
import webbrowser

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vc\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [10]:
A = wn.synsets('mercator')
A[0].hypernyms()
#A[0].hypernyms()[0].lemma_names()

[]

In [126]:
A[0].hypernyms()[0].lemma_names()

['faculty', 'mental_faculty', 'module']

In [106]:
wn.synsets('fender')[1].lemma_names()

['fender', 'buffer', 'cowcatcher', 'pilot']

In [12]:
def extract_hypernym(query_term):
    hypernym_list = {}
    contexts = {}
    for i, element in enumerate(wn.synsets(query_term)):
        temp = element.hypernyms()[0].lemma_names()
        contexts[i] = temp 
    return contexts


In [13]:
extract_hypernym('peacock')

{0: ['nymphalid',
  'nymphalid_butterfly',
  'brush-footed_butterfly',
  'four-footed_butterfly'],
 1: ['peafowl', 'bird_of_Juno']}

In [19]:
def extract_synonyms(query_term):
    temp = {}
    contexts = {}
    for i, element in enumerate(wn.synsets(query_term)):
        temp = element.lemma_names()
        contexts[i] = temp
    return contexts

In [66]:
extract_synonyms("cacophony")

{0: ['blare', 'blaring', 'cacophony', 'clamor', 'din'], 1: ['cacophony']}

In [14]:
threshold =0.1
idf=pd.read_pickle(r'./Storage/inv_doc_freq.pkl')

In [9]:
print(max(idf.values()))

2.6483600109809315


In [79]:
query = input('Type the query:')
query = query.lower()
query=query.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
query=query.translate(str.maketrans("‘’’–——−",'       '))
query_words = []
query_words = word_tokenize(query)
query_words=list(set(query_words))
query_words

Type the query:top monument


['top', 'monument']

# Starts from here

In [80]:
class WordnetImprovement:
    def __init__(self, query_term):
        self.term = query_term
        
        
    def extract_hypernyms(self):
        hypernym_list = {}
        contexts = {}
        for i, element in enumerate(wn.synsets(self.term)):
            temp = element.hypernyms()[0].lemma_names()
            contexts[i] = temp 
        return contexts


    def extract_synonyms(self):
        temp = {}
        contexts = {}
        for i, element in enumerate(wn.synsets(self.term)):
            temp = element.lemma_names()
            contexts[i] = temp
        return contexts

In [76]:
'''
alt_query_dict={}

for token in query_words:
    syn ={}
    hyp = {}
    if idf.get(token) != None and idf[token]>threshold:
        hyp = WordnetImprovement.extract_hypernyms(token)[0]
        syn = WordnetImprovement.extract_synonyms(token)[0]

for n in range(len(syn)):
    temp = query_words.copy()
    for i in range(len(temp)):
        if idf[temp[i]]>threshold:
            idx = i
    if temp[idx] != hyp[n]:
        temp[idx] = hyp[n]
        alt_query_dict[n] = temp

alt_query_dict
'''
print('hmm')

hmm


In [83]:
def query_relaxation(tokenized_query, mode = 'hypernym'):
    threshold = 1
    parallel_query_dict = {}
    syn = {}
    hyp = {}
    temp = []
    for token in tokenized_query:
        if idf.get(token) != None and idf[token] > threshold:
            relaxer = WordnetImprovement(token)
            hyp = relaxer.extract_hypernyms()[0]
            syn = relaxer.extract_synonyms()[0]
            
            
    if mode == 'hypernym':
        for n in range(len(hyp)):
            temp = tokenized_query.copy()
            for i in range(len(temp)):
                if idf[temp[i]]>threshold:
                    idx = i
            if temp[idx] != hyp[n]:
                temp[idx] = hyp[n]
                parallel_query_dict[n] = temp
                
                
    elif mode == 'synonym':
        for n in range(len(syn)):
            temp = tokenized_query.copy()
            for i in range(len(temp)):
                if idf[temp[i]]>threshold:
                    idx = i
            if temp[idx] != syn[n]:
                temp[idx] = syn[n]
                parallel_query_dict[n] = temp
    return parallel_query_dict


In [84]:
parallel_queries = query_relaxation(query_words)
parallel_queries

{0: ['top', 'structure'], 1: ['top', 'construction']}

In [63]:
parallel_query_scores = {}
for n, query in parallel_queries.items():
    parallel_query_scores[n] = query_tester.find_relevant(query)
parllel_query_scores

NameError: name 'query_tester' is not defined